In [ ]:
from caffe2.python import (
    core,
)

from caffe2.python.fb.dper.layer_models.models import sparse_nn
from fblearner.flow.projects.dper.preprocs.ads import build_preproc
from fblearner.flow.projects.dper.preprocs.ads_feature_processor import (
    ads_feature_processor,
)
from hiveio import par_init  # noqa
import fblearner.flow.projects.dper.flow_types as T
import fblearner.flow.projects.dper.utils.assemble as assemble_utils
import fblearner.flow.projects.dper.utils.data as data_utils
import fblearner.flow.projects.dper.utils.visualize as vis_utils
import fblearner.flow.projects.dper.workflows.ads_config as default_config

import fblearner.flow.projects.dper.ifbpy.compute_meta as compute_meta
from fblearner.flow.projects.dper.ifbpy.execution import test_model_locally
import fblearner.flow.projects.dper.utils.visualize as vis_utils
import fblearner.flow.projects.dper.utils.perf_estimator_execution as perf_estimator_execution

import json
core.GlobalInit(['ifbpy'])
from IPython.core.debugger import Pdb; 
ipdb=Pdb()

In [ ]:
# when testing a particular flow, load model options from json file, and pass it to model_options
# local_prod_jason_file="/home/dongli/fbsource/fbcode/caffe2/caffe2/net_config/33252482/prod_model.json"
# with open(local_prod_jason_file, 'r') as f:
#     prod_model_options = sparse_nn.MODEL_OPTIONS.decode(json.loads(f.read()))
# print(prod_model_options)        



In [ ]:
preproc_options = default_config.DEFAULT_PREPROC_OPTIONS

# when testing a particular flow, load model options from json file
# load model preproc options from json file
# from fblearner.flow.projects.dper.preprocs.ads import build_preproc
# local_prod_preproc_jason_file="/home/dongli/fbsource/fbcode/caffe2/caffe2/net_config/33252482/prod_preproc.json"
# with open(local_prod_preproc_jason_file, 'r') as f:
#     prod_preproc_options = build_preproc.options_flow_type.decode(json.loads(f.read()))
# print prod_preproc_options

# preproc_options = prod_preproc_options

In [ ]:
# Finalize config for preprocessor
compute_meta.resolve_compute_meta(ads_feature_processor, default_config.DEFAULT_DATASET, preproc_options)
print("Done: resolve_compute_meta")

In [ ]:
# Assemble the model given preprocessor and model building fuction
model = assemble_utils.assemble_model(
        name='sparse_nn',
        input_feature_schema=build_preproc.input_feature_schema(
            preproc_options),
        trainer_extra_schema=build_preproc.trainer_extra_schema(
            preproc_options),
        build_preproc_fun=build_preproc,
        build_model_fun=sparse_nn.build_model,
        preproc_options=preproc_options,
        model_options= default_config.DEFAULT_MODEL_OPTIONS
)

In [ ]:
# Train model one the given sample dataset
estimated_cost = perf_estimator_execution.estimate_perf_locally(model, default_config.DEFAULT_DATASET)
print(estimated_cost)